In [1]:
import kaggle
import pandas as pd
#import libraries

In [17]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv --force
#download dataset orders.csv.zip

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 91.0MB/s]


In [23]:
#extract from zip file
import zipfile

zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [25]:
df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
df.head(20)
df['Ship Mode']

#Not Available,unknown to be treated as null

df.columns = df.columns.str.lower() #replace(' ','_')
df.columns = df.columns.str.replace(' ','_')

#column names set according to SQL table column names

In [33]:
#derive new columns discount , sale price and profit

df.head(5)
#df['discount'] = df['list_price']*df['discount_percent']/100
df['sale_price'] = df['list_price']-df['discount']
df['profit'] = (df['list_price']-df['discount']) - df['cost_price']


In [37]:
#changing format of order_date column

df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [43]:
#drop cost price list price and discount percent columns
df.drop(axis=1,columns=['cost_price','list_price','discount_percent'],inplace = True)

In [45]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0


In [73]:
#load the data into sql server using replace option
import sqlalchemy as sal
import urllib.parse
import configparser

config = configparser.ConfigParser()
config.read(r"config.ini")

user = config["postgre_database"]["user"]
password = config['postgre_database']['password']
host = config['postgre_database']['host']
port = config['postgre_database']['port']
db = config['postgre_database']['db']

encoded_password = urllib.parse.quote_plus(password)
engine = sal.create_engine(f"postgresql+psycopg2://{user}:{encoded_password}@{host}:{port}/{db}")

# Define the CREATE TABLE statement
create_table_sql = """
CREATE TABLE IF NOT EXISTS df_orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    ship_mode VARCHAR(20),
    segment VARCHAR(20),
    country VARCHAR(20),
    city VARCHAR(20),
    state VARCHAR(20),
    postal_code VARCHAR(20),
    region VARCHAR(20),
    category VARCHAR(20),
    sub_category VARCHAR(20),
    product_id VARCHAR(50),
    quantity INT,
    discount DECIMAL(7,2),
    sale_price DECIMAL(7,2),
    profit DECIMAL(7,2)
);
"""

with engine.begin() as conn:  # ensures commit
    conn.execute(text(create_table_sql))
    df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')
